In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import LineString

In [2]:
depot_long = -73.941868
depot_lat = 40.725516
df_depot = pd.DataFrame({'id':'Depot', 'lat': [depot_lat], 'long': [depot_long]})
df_depot

,id,lat,long
0,Depot,40.725516,-73.941868


In [3]:
shops_2015 = pd.read_csv('2015_shop_locations.csv')
shops_2015 = pd.concat([df_depot, shops_2015], ignore_index=True)
shops_2015

,id,lat,long,demand(kg),stage
0,Depot,40.725516,-73.941868,NaN,NaN
1,ChIJwTC7A55ZwokRPNX9g7ngbaI,40.730837,-73.983194,21.0,1.0
2,ChIJU1XImaNZwokRutunetC8XeE,40.740454,-73.991268,33.0,1.0
3,ChIJG7L-TLVbwokRT36uIrwz2Mo,40.694957,-73.982865,21.0,1.0
4,ChIJ5cPkuBtgwokRn55JgpGqjFA,40.757522,-73.834352,30.0,1.0
...,...,...,...,...,...
216,ChIJ3wsoxKFZwokRRg8OvEjE0bc,40.736716,-73.986817,19.0,4.0
217,ChIJQRpR6Jj0wokRl9myJ19eEvU,40.850804,-73.867450,28.0,4.0
218,ChIJQYmJtx5awokR45_sopkXVNE,40.715137,-74.011088,32.0,4.0
219,ChIJh4MG8h9awokR1kDLslu2C3E,40.718249,-74.007346,33.0,4.0


In [4]:
shops_2020 = pd.read_csv('2020_shop_locations.csv')
shops_2020 = pd.concat([df_depot, shops_2020], ignore_index=True)
shops_2020.head()

,id,lat,long,Unnamed: 0,demand(kg),stage,borought,zone,location_id
0,Depot,40.725516,-73.941868,NaN,NaN,NaN,NaN,NaN,NaN
1,ChIJwTC7A55ZwokRPNX9g7ngbaI,40.730837,-73.983194,0.0,21.0,1.0,Manhattan,East Village,79.0
2,ChIJU1XImaNZwokRutunetC8XeE,40.740454,-73.991268,1.0,33.0,1.0,Manhattan,Union Sq,234.0
3,ChIJG7L-TLVbwokRT36uIrwz2Mo,40.694957,-73.982865,2.0,21.0,1.0,Brooklyn,Downtown Brooklyn/MetroTech,65.0
4,ChIJ5cPkuBtgwokRn55JgpGqjFA,40.757522,-73.834353,3.0,30.0,1.0,Queens,Flushing,92.0


## Function for creating a dataframe for both source and destination points in the results (from or to)

In [5]:
def resultsCleaning(maindf, shopsdf):    
    maindf['f-lat'] = maindf['From'].iloc[:].apply(lambda x: shopsdf[shopsdf['id'] == x]['lat'].values[0])
    maindf['f-long'] = maindf['From'].iloc[:].apply(lambda x: shopsdf[shopsdf['id'] == x]['long'].values[0])

    maindf['t-lat'] = maindf['To'].iloc[:].apply(lambda x: shopsdf[shopsdf['id'] == x]['lat'].values[0])
    maindf['t-long'] = maindf['To'].iloc[:].apply(lambda x: shopsdf[shopsdf['id'] == x]['long'].values[0])

    maindf = maindf[['From', 'f-lat', 'f-long', 'To', 't-lat', 't-long']]

    df1 = maindf[['From', 'f-lat', 'f-long']]
    df2 = maindf[['To', 't-lat', 't-long']]
    
    return df1, df2

## Function for creating and plotting the map

In [6]:
def routemap(maindf, shopsdf, dfFrom, dfTo):
    from_gdf = gpd.GeoDataFrame(dfFrom, geometry=gpd.points_from_xy(dfFrom['f-long'], dfFrom['f-lat']))
    to_gdf = gpd.GeoDataFrame(dfTo, geometry=gpd.points_from_xy(dfTo['t-long'], dfTo['t-lat']))

    from_gdf['Legend'] = dfFrom['From'].apply(lambda x: 'Depot' if x == 'Depot' else '')

    # We remove Depot from the second gdf so that it doesn't appear twice on the map
    to_gdf = to_gdf[to_gdf['To'] != 'Depot']
    to_gdf['Legend'] = dfTo['To'].apply(lambda x: 'Depot' if x == '' else 'Shop')

    # We need to rename the From and To columns to the same name so that we can concatenate the two dataframes and plot the hover names using just this column
    from_gdf = from_gdf.rename(columns={'From': 'id'})
    to_gdf = to_gdf.rename(columns={'To': 'id'})

    combined_gdf = pd.concat([from_gdf, to_gdf])

    # Size of the points
    combined_gdf['size'] = 1

    # Create the route connections between all of the depots
    lines = []
    for i, row in maindf.iterrows():
        from_id = row['From']
        to_id = row['To']

        from_coords = (shopsdf[shopsdf['id'] == from_id]['lat'].values, shopsdf[shopsdf['id'] == from_id]['long'].values)
        to_coords = (shopsdf[shopsdf['id'] == to_id]['lat'].values, shopsdf[shopsdf['id'] == to_id]['long'].values)

        line = LineString([from_coords, to_coords])
        lines.append(line)

    lines_gdf = gpd.GeoDataFrame(geometry=lines)

    # Plot the map itself
    fig = px.scatter_mapbox(
        combined_gdf,
        lat=combined_gdf.geometry.y,
        lon=combined_gdf.geometry.x,
        color='Legend',
        hover_name='id',
        size='size',
        size_max=10,
        zoom=10.5,
        mapbox_style="carto-positron",
        color_discrete_map={'Depot': 'red', 'Shop': 'green'}
    )

    # Add the lines (LineStrings) to the map
    for line in lines_gdf.geometry:
        # Convert each LineString into a list of lat-lon coordinates
        line_coords = [(point[1], point[0]) for point in line.coords]
        
        # Add the line trace to the map
        fig.add_trace(
            go.Scattermapbox(
                mode='lines',
                lon=[coord[0] for coord in line_coords],
                lat=[coord[1] for coord in line_coords],
                line={'width': 2, 'color': 'green'},
            )
        )

    # Modify the layout margins and the map height
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        height=800,  # Height of the map (in pixels)
    )

    fig.show()

# Application on 2015 data

## Stage 1

In [7]:
df_trips = pd.read_csv('routings/routing_optimal_2015_stage_1_Distance.csv')
df_trips.head()

,From,To
0,Depot,ChIJ97BgiDtfwokRsbkP6Vl1seY
1,Depot,ChIJmZQsMgFbwokRMc5I7kPdQ7w
2,ChIJwTC7A55ZwokRPNX9g7ngbaI,Depot
3,ChIJU1XImaNZwokRutunetC8XeE,ChIJwTC7A55ZwokRPNX9g7ngbaI
4,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJRR7W62BFwokRlrVks40mAdU


In [8]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

## Stage 2

In [9]:
df_trips = pd.read_csv('routings/routing_optimal_2015_stage_2_Distance.csv')
df_trips.head()

,From,To
0,Depot,ChIJiUJ1DI5ZwokRWdK6SPg9BOY
1,Depot,ChIJzRG6yrrzwokRQ5kjG2RJCwM
2,Depot,ChIJ0zAa911ZwokRQlmIU8v4Kzs
3,Depot,ChIJLXqvAAJdwokRSeu3qp3b1es
4,Depot,ChIJ_QfagF9cwokR7b8aZshUano


In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

## Stage 3

In [ ]:
df_trips = pd.read_csv('routings/routing_optimal_2015_stage_3_Distance.csv')
df_trips.head()

In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

## Stage 4

In [ ]:
df_trips = pd.read_csv('routings/routing_optimal_2015_stage_4_Distance.csv')
df_trips.head()

In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

# Application on 2020 data

## Stage 1

In [ ]:
df_trips = pd.read_csv('routings/routing_optimal_2020_stage_1_Distance.csv')
df_trips.head()

In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

## Stage 2

In [ ]:
df_trips = pd.read_csv('routings/routing_optimal_2020_stage_2_Distance.csv')
df_trips.head()

In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

## Stage 3

In [ ]:
df_trips = pd.read_csv('routings/routing_optimal_2020_stage_3_Distance.csv')
df_trips.head()

In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)

## Stage 4

In [ ]:
df_trips = pd.read_csv('routings/routing_optimal_2020_stage_4_Distance.csv')
df_trips.head()

In [ ]:
# Adding latitude and longitude from the shops dataframe
df_trips_from, df_trips_to = resultsCleaning(df_trips, shops_2015)

# Creating and plotting the map
routemap(df_trips, shops_2015, df_trips_from, df_trips_to)